### Set up pipeline with basic imports, data directories, and file basenames.

In [ ]:
import os
from benbiohelpers.DataPipelineManagement.GenomeManager import getGenomeFastaFilePath
from bpde_chromatin_analysis.helper_scripts.BPDE_DataDir import getDataDir
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import getExternalDataDirectory as getMutperiodExternalDataDirectory

hg19FastaFilePath = getGenomeFastaFilePath("hg19")
mutperiodHg19Directory = getMutperiodExternalDataDirectory()

liDataDirectory = os.path.join(getDataDir(), "Li_tXR-seq")
liBaseDataFilePath = os.path.join(liDataDirectory, "Li_tXR-seq.bed")

jiang2uM_CellularDataDir = os.path.join(getDataDir(), "Jiang_BPDE_damage_maps", "BEAS-2B_2uM_BPDE_cell_24h")

### Parse the tXR-seq data for mutperiod input

In [ ]:
from mutperiodpy.input_parsing.ParseXRSeq import parseXRSeq, PresetCallParams

liMutperiodInputFilePaths = parseXRSeq([liBaseDataFilePath], PresetCallParams.BPDE, hg19FastaFilePath)

### Run the mutperiod pipeline to determine translational and rotational periodicity of the data.
NOTE: This analysis requires that the damage data has already been parsed for mutperiod using the Jiang_damage_maps_nucleosome_periodicity_data_generation notebook.

In [ ]:
from mutperiodpy.RunAnalysisSuite import runAnalysisSuite
nucleosomeMapFilePaths = list()
nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_hybrid_nucleosome_map", "hg19_hybrid_nucleosome_map.bed"))
nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_LCL_MNase_nuc_map_all_mappable", "hg19_LCL_MNase_nuc_map_all_mappable.bed"))
nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_LCL_MNase_nuc_map_all_mappable_euchromatin",
                                           "hg19_LCL_MNase_nuc_map_all_mappable_euchromatin.bed"))
nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_LCL_MNase_nuc_map_all_mappable_heterochromatin",
                                           "hg19_LCL_MNase_nuc_map_all_mappable_heterochromatin.bed"))
nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_LCL_MNase_nuc_map_all_mappable_stringent_euchromatin",
                                           "hg19_LCL_MNase_nuc_map_all_mappable_stringent_euchromatin.bed"))
nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_NHF1_MNase_nuc_map_mappable", "hg19_NHF1_MNase_nuc_map_all_mappable.bed"))
nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_CTCF_known_all", "hg19_CTCF_known_all.bed"))

nucleosomeMapNames = [os.path.basename(nucleosomeMapFilePath).rsplit('.', 1)[0] for nucleosomeMapFilePath in nucleosomeMapFilePaths]

runAnalysisSuite(liMutperiodInputFilePaths, nucleosomeMapNames,
                 normalizationMethod = "No Normalization", customBackgroundDir = None,
                 useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True)
runAnalysisSuite(liMutperiodInputFilePaths, nucleosomeMapNames,
                 normalizationMethod = "Singlenuc/Dinuc", customBackgroundDir = None,
                 useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True)
runAnalysisSuite(liMutperiodInputFilePaths, nucleosomeMapNames,
                 normalizationMethod = "Trinuc/Quadrunuc", customBackgroundDir = None,
                 useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True)
runAnalysisSuite(liMutperiodInputFilePaths, nucleosomeMapNames,
                 normalizationMethod = "Custom Background", customBackgroundDir = jiang2uM_CellularDataDir,
                 useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True)

In [ ]:
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import DataTypeStr
from mutperiodpy.RunNucleosomeMutationAnalysis import runNucleosomeMutationAnalysis
from benbiohelpers.FileSystemHandling.DirectoryHandling import getFilesInDirectory

runNucleosomeMutationAnalysis(getFilesInDirectory(liDataDirectory, DataTypeStr.rawNucCounts + ".tsv"),
                              outputFilePath = os.path.join(liDataDirectory, "Li_tXR-seq_raw_periodicity_data.tsv"),
                              overridePeakPeriodicityWithExpected = False, alignStrands = True)
runNucleosomeMutationAnalysis(getFilesInDirectory(liDataDirectory, DataTypeStr.normNucCounts + ".tsv"),
                              outputFilePath = os.path.join(liDataDirectory, "Li_tXR-seq_damage_normalized_periodicity_data.tsv"),
                              overridePeakPeriodicityWithExpected = False, alignStrands = True)